In [ ]:
import os
import sys
import asyncio
import logging
import time

import numpy as np

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

import lsst.observing.utils.focus
from lsst.observing.constants import (boreSight, sweetSpots, gratingOffsets,
                                      dFocusDthickness, glassThicknesses, plateScale)
from lsst.observing.utils.misc import parseObsId
from lsst.observing.utils.audio import playSound
from lsst.observing.utils.offsets import findOffsetsAndMove
from lsst.observing.utils.filters import getFilterAndGrating, changeFilterAndGrating

Make sure that logs aren't lost

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.ERROR

In [ ]:
if True:
    attcs = ATTCS()
    latiss = LATISS(attcs.domain)

Execute this cell if you want to pass `display=display` to `findOffsetsAndMove`

In [ ]:
if False:
    import os
    import lsst.afw.display as afwDisplay

    afwDisplay.setDefaultBackend("matplotlib" if False else "firefly")
    os.environ['FIREFLY_URL'] = "http://firefly01.cp.lsst.org:8080/firefly/"

    display = afwDisplay.Display(1, name='RHL', reopenPlot=True)

    display.getClient().display_url()

In [ ]:
for repo in ["/mnt/dmcs/oods_butler_repo/repo",
             "/project/shared/auxTel",
             os.path.expanduser("~/Data/auxTel" "XX"),
            ]:
    if os.path.exists(repo):
        break

print(f"Using {repo}")  
repo = os.path.join(repo, "rerun", "quickLook")

if os.path.exists(repo):
    from lsst.daf.persistence import Butler
else:
    print(f"I can't find {repo}; using butler emulator")
    from lsst.ts.standardscripts.auxtel.butler import Butler
    
    latiss.domain.time_per_second = 0.0

butler = Butler(repo)
dataId = dict(dayObs="2020-02-21")

In [ ]:
repo = "/mnt/dmcs/oods_butler_repo/repo/rerun/quickLook"

if os.path.exists(repo):
    from lsst.daf.persistence import Butler
else:
    print(f"I can't find {repo}; using emulator")
    from lsst.ts.standardscripts.auxtel.butler import Butler
    
    latiss.domain.time_per_second = 0.0

butler = Butler(repo)
dataId = dict(dayObs="2020-02-21")

<HR>

Master options.  Will be arguments some day

In [ ]:
if True:
    objectName = "HD 185975"    # pole
else:
    objectName = "HD 146233"

chosenGrating = 'ronchi90lpmm'  # the grating to use

doPointingModel = True
expTime0 = 1.0                 # initial guess at the unit of exposure time

lsst.observing.utils.focus.myFocusOffset = None

From https://confluence.lsstcorp.org/display/LSSTCOM/2020-02-20

- a. Slew to object with Parallactic compensation on 

In [ ]:
pa_angs = [0, -180]

for i, pa_ang in enumerate(pa_angs):
    try:
        await asyncio.gather(
            attcs.slew_object(name=objectName, pa_ang=pa_ang, slew_timeout=240),
            latiss.setup_atspec(grating='empty_1', filter='empty_1'),
        )
    except Exception as e:
        if i == 0:
            print(f"Failed to go to {pa_ang} ({e}); trying {pa_angs[1]}")
        else:
            raise
    else:
        print(f"pa = {pa_ang}")
        break

playSound()

We are now nominally at the boresight.  We need to check this if we're updating the pointing model

if `doPointingModel` is true:

- b. take test image, determine offset to boresight
- c. sanity-check the computed offsets
- d. do x,y offset to center object at boresight

In [ ]:
if doPointingModel:
    targetPosition = boreSight

    retvals = await asyncio.gather(
        findOffsetsAndMove(attcs, targetPosition, latiss, dataId=None, butler=butler,
                           doMove=True, alwaysAcceptMove=True),
        latiss.take_object(exptime=expTime0, n=1),
    )
    exp, dx, dy, peakVal = retvals[0]
    
    print(f"Peak: {peakVal}")
    playSound()

if `doPointingModel` is true:

- d'. Check that we're at the boresight

In [ ]:
if doPointingModel:
    await latiss.take_object(exptime=expTime0, n=1)
    
    playSound()

e. update pointing model

N.b. If `doPointingModel` and we don't have a pointing model, exec the next cell

In [ ]:
if False:
    await attcs.atptg.cmd_pointNewFile.start()

In [ ]:
if doPointingModel:
    await attcs.atptg.cmd_pointAddData.start()

Done with pointing model.

Offset to sweet spot with grating correction (so the star will be in the wrong place as we don't have
a grating in)

f. take a test exposure and check flux levels, determine best exptime for `chosenGrating`

N.b. update expTime0 -- need code to find peak flux in object

In [ ]:
targetPosition = sweetSpots[chosenGrating]

retvals = await asyncio.gather(
    findOffsetsAndMove(attcs, targetPosition, latiss, dataId=None, butler=butler, 
                       doMove=True, alwaysAcceptMove=True, display=None),
    latiss.take_object(exptime=expTime0, n=1, grating=chosenGrating),
)
exp, dx, dy, peakVal = retvals[0]
print(f"Peak = {peakVal:0f}")

playSound()

Run focus sweep and measure desired focus offset

In [ ]:
while True:
    reply = input("What is the best focus offset for the current configuration (mm)? ")
    try:
        focus_offset = float(reply)
    except Exception as e:
        print(f"Error ({e}).  Please try again:")
    else:
        break

if False:
    attcs.athexapod.evt_positionUpdate.flush()
else:
    await asyncio.sleep(5)

await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_offset) 
await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)

lsst.observing.utils.focus.myFocusOffset = focus_offset

playSound()

At this point the grating is in and the filter is empty

Ready for science!  First spectroscopy:

In [ ]:
expTime = 45*expTime0

await changeFilterAndGrating(attcs, latiss, filter='empty_1', grating=chosenGrating)
await latiss.take_object(exptime=expTime, n=1)
playSound("ding")

await changeFilterAndGrating(attcs, latiss, filter='quadnotch1')
await latiss.take_object(exptime=expTime, n=1)
playSound("ding")

await changeFilterAndGrating(attcs, latiss, filter='BG40')
await latiss.take_object(exptime=expTime, n=1)
playSound("ding")

await changeFilterAndGrating(attcs, latiss, filter='RG610')
await latiss.take_object(exptime=expTime, n=1)

playSound()

And then photometry

In [ ]:
for i, f in enumerate(['BG40', 'RG610']):
    await changeFilterAndGrating(attcs, latiss, filter=f, grating='empty_1')

    await latiss.take_object(exptime=3*expTime0, n=1)
    playSound("ding")

    await latiss.take_object(exptime=20*expTime0, n=1)
    playSound(None if i == 0 else "gong")